---
output-file: config.html
title: Configuration
---

In [ ]:
#| default_exp config

## Configuration Management

The configuration module handles persistent storage of user preferences and API credentials. It follows the XDG Base Directory Specification for storing configuration files in standard system locations.

### Key Features

- **XDG Compliance**: Stores configuration in `~/.config/tui_writer/` on Linux/macOS <br>
- **Multi-Provider API Keys**: Secure storage for OpenAI, Anthropic, and Google Gemini API keys <br>
- **Session Persistence**: Remembers the last selected provider and model <br>
- **Type-Safe Configuration**: Uses dataclasses with type hints for configuration  structure <br>
- **Auto-Creation**: Automatically creates config directory and file on first run <br>

### Configuration File Location

The configuration file is stored at:
```
{XDG_CONFIG_HOME}/tui_writer/tui_writer.conf
```

On most systems, this resolves to `~/.config/tui_writer/tui_writer.conf`.


In [ ]:
#| export
#| include: false

from dataclasses import dataclass
from fastcore.all import *
from fastcore.xdg import *
from typing import get_type_hints

In [ ]:
#| export
#| include: false
_tui_writer_home_dir = 'tui_writer'
_tui_writer_cfg_name = 'tui_writer.conf'

In [ ]:
#| export
#| include: false
def _cfg_path(): return xdg_config_home() / _tui_writer_home_dir / _tui_writer_cfg_name

In [ ]:
#| include: false
_cfg_path()

## TuiWriterConfig Dataclass

The configuration is structured as a dataclass with five fields: <br>

- **openai_key**: API key for OpenAI models (GPT-4, GPT-3.5, etc.) <br>
- **anthropic_key**: API key for Anthropic models (Claude) <br>
- **gemini_key**: API key for Google Gemini models <br>
- **last_provider**: The most recently selected provider (openai/anthropic/gemini) <br>
- **last_model**: The most recently selected model ID <br>

All fields default to empty strings. The configuration system handles missing or incomplete configurations gracefully.

In [ ]:
#| export
#| include: false
@dataclass
class TuiWriterConfig:
    openai_key: str = ''
    anthropic_key: str = ''
    gemini_key: str = ''
    last_provider: str = ''
    last_model: str = ''
    transcribe_only: bool = False
    rec_on_launch: bool = True

In [ ]:
#| include: false
cfg = TuiWriterConfig()
cfg

In [ ]:
#| export
#| include: false
def get_cfg():
    path = _cfg_path()
    path.parent.mkdir(parents=True, exist_ok=True)
    _types = get_type_hints(TuiWriterConfig)
    return Config(
        path.parent,
        path.name,
        create=asdict(TuiWriterConfig()),
        types=_types,
        inline_comment_prefixes=('#')
    )

## get_cfg Function

The `get_cfg()` function is the main interface for accessing configuration. It returns a `Config` object from fastcore that provides dictionary-like access to configuration values with automatic file synchronization.

**Behavior:**
- Creates the configuration directory if it doesn't exist
- Creates the configuration file with default values on first run
- Returns a `Config` instance with type checking based on `TuiWriterConfig` hints
- Supports inline comments with `#` prefix
- Automatically saves changes back to the file

**Usage:**
```python
cfg = get_cfg()
cfg['openai_key'] = 'sk-...'
cfg['last_provider'] = 'openai'
cfg['last_model'] = 'gpt-4o-mini'
```

The configuration is immediately persisted to disk when values are modified.